Для 3D был выбран 5ый вариант, в соответствием со списком на AnyTask. 
Как видим из результатов, алгоритм на иерархической очереди работает заметно быстрее, чем алгоритм на приоритетной очереди. Отметим также, что для 3D изображения была найдена только одна область ввиду определенного выбора точек, которые можно изменить в списке <i> points</i>.

In [ ]:
%matplotlib inline
import numpy as np
from skimage import io as skio
from collections import deque
from datetime import datetime
from skimage import color
from matplotlib import pyplot as plt
from heapq import heappush, heappop, nsmallest


def CalculateGrad2d(img):
    grad = np.gradient(img)
    grad = np.array([1000 * (grad[0][x, y] ** 2 + grad[1][x, y] ** 2) ** 0.5 
                    for x in range(img.shape[0])
                   for y in range(img.shape[1])]).reshape(img.shape).astype(np.int32)
    return grad


def CalculateWatershedPrior2D(grad, points):
    q = []
    counter = 0
    flag = np.zeros(grad.shape).astype(np.bool)
    mask = np.zeros(grad.shape).astype(np.int)    
    steps = [(1, 0), (0, 1), (-1, 0), (0, -1)]
    
    for i, j in enumerate(points):
        flag[j] = True
        mask[j] = i + 1
        heappush(q, ((grad[j], counter), j))
        counter += 1

    while len(q) != 0:
        x, y = heappop(q)[1]
        for st in steps:
            cx = x + st[0]
            cy = y + st[1]
            if 0 <= cx < grad.shape[0] and 0 <= cy < grad.shape[1]:
                if not flag[cx, cy]:
                    mask[cx, cy] = mask[x, y]  
                    flag[cx, cy] = True
                    heappush(q, ((grad[cx, cy], counter), (cx, cy)))
                    counter += 1
    return mask


def CalculateWatershedHier2D(grad, points):
    flag = np.zeros(grad.shape).astype(np.bool)
    mask = np.zeros(grad.shape).astype(np.int)
    queues = [deque() for i in range(np.max(grad) + 1)]
    steps = [(1, 0), (0, 1), (-1, 0), (0, -1)]

    for i, j in enumerate(points):
        flag[j] = True
        mask[j] = i + 1
        queues[grad[j]].append(j)

    for i in range(len(queues)):
        q = queues[i]
        while len(q) != 0:
            x, y = q.popleft()
            for st in steps:
                cx = x + st[0]
                cy = y + st[1]
                if 0 <= cx < grad.shape[0] and 0 <= cy < grad.shape[1]:
                    if not flag[cx, cy]:
                        mask[cx, cy] = mask[x, y]
                        flag[cx, cy] = True
                        queues[max(grad[cx, cy], i)].append((cx, cy))
    return mask


def ShowImage(img):
    plt.figure(figsize=(10, 8))
    plt.axis('off')
    skio.imshow(img)
    plt.plot()
    skio.show()
    

def CalculateGrad3D(img):
    grad = np.gradient(img)
    grad = np.array([(grad[0][x, y, z] ** 2 + grad[1][x, y, z] ** 2 + grad[2][x, y, z] ** 2) ** 0.5
                         for x in range(img.shape[0])
                         for y in range(img.shape[1])
                         for z in range(img.shape[2])]).reshape(img.shape).astype(np.int)
    return grad


def CalculateWatershedPrior3D(grad, points):
    q = []
    counter = 0
    mask = np.zeros(grad.shape).astype(np.int)
    flag = np.zeros(grad.shape).astype(np.bool)    
    steps = [(1, 0, 0), (0, 1, 0), (0, 0, 1), (-1, 0, 0), (0, -1, 0), (0, 0, -1)]
  
    for i, j in enumerate(points):     
        flag[j] = True
        mask[j] = i + 1
        heappush(q, ((grad[j], counter), j))
        counter += 1

    while len(q) != 0:
        x, y, z = heappop(q)[1]
        for st in steps:
            cx = x + st[0]
            cy = y + st[1]
            cz = z + st[2]
            if 0 <= cx < grad.shape[0] and 0 <= cy < grad.shape[1] and 0 <= cz < grad.shape[2]:
                if not flag[cx, cy, cz]:
                    mask[cx, cy, cz] = mask[x, y, z]  
                    flag[cx, cy, cz] = True
                    heappush(q, ((grad[cx, cy, cz], counter), (cx, cy, cz)))
                    counter += 1
    return mask


def CalculateWatershedHier3D(grad, points):  
    flag = np.zeros(grad.shape).astype(np.bool)
    mask = np.zeros(grad.shape).astype(np.int)
    queues = [deque() for i in range(np.max(grad) + 1)]    
    steps = [(1, 0, 0), (0, 1, 0), (0, 0, 1), (-1, 0, 0), (0, -1, 0), (0, 0, -1)]

    for i, j in enumerate(points):  
        flag[j] = True
        mask[j] = i + 1
        queues[grad[j]].append(j)

    for i in range(len(queues)):
        q = queues[i]
        while len(q) != 0:
            x, y, z = q.popleft()
            for st in steps:
                cx = x + st[0]
                cy = y + st[1]
                cz = z + st[2]
                if 0 <= cx < grad.shape[0] and 0 <= cy < grad.shape[1] and 0 <= cz < grad.shape[2]:
                    if not flag[cx, cy, cz]:
                        mask[cx, cy, cz] = mask[x, y, z]
                        flag[cx, cy, cz] = True
                        queues[max(grad[cx, cy, z], i)].append((cx, cy, cz))                  
    return mask

In [ ]:
print("----------------------------2D-----------------------------")
crcl = color.rgb2gray(skio.imread("C:\\Users\\VaitsiakhouskiA\\Downloads\\circles.png"))
ShowImage(crcl)
grad = CalculateGrad2d(crcl)
print("Gradient:")
ShowImage(grad)

print("Prioritet watershed 2D:")
points = [(250, 250), (200, 250), (150, 250), (100, 250), (50, 250), (0, 250)]
start = datetime.now()
ShowImage(CalculateWatershedPrior2D(grad, points))
end = datetime.now()
print(end - start)

print("Hierarchical watershed 2D:")
points = [(250, 250), (200, 250), (150, 250), (100, 250), (50, 250), (0, 250)]
start = datetime.now()
ShowImage(CalculateWatershedHier2D(grad, points))
end = datetime.now()
print(end - start)


print("----------------------------3D-----------------------------")
l = 0
h = 100
mid = (h - l) / 2
img = np.fromfile("C:\\Users\\VaitsiakhouskiA\\Downloads\\id005\\id005.raw", dtype=np.int16).reshape(512, 512, 100)[:, :, l:h]
ShowImage(img[:, :, mid])
ShowImage(img[:, 200, :])
ShowImage(img[200, :, :])

print("Gradient:")
grad = CalculateGrad3D(img)
ShowImage(grad[:, :, mid])
ShowImage(grad[:, 200, :])
ShowImage(grad[200, :, :])

print("Prioritet watershed:")
points = [(10, 250, mid), (150, 250, mid), (200, 450, mid), (200, 100, mid)]
start = datetime.now()
waterShed = CalculateWatershedPrior3D(grad, points)
end = datetime.now()
print(end - start)
ShowImage(waterShed[:, :, mid])
ShowImage(waterShed[:, 200, :])
ShowImage(waterShed[200, :, :])

print("Hierarchical watershed:")
start = datetime.now()
waterShed = CalculateWatershedHier3D(grad, points)
end = datetime.now()
print(end - start)
ShowImage(waterShed[:, :, mid])
ShowImage(waterShed[:, 200, :])
ShowImage(waterShed[200, :, :])